# <center>Analysis of World of Warcraft Population Data</center>
### <center>Data obtained from [realmpop.com](https://realmpop.com/)</center>

This blog post uses names and images from World of Warcraft and data proprietary to World of Warcraft. World of Warcraft, Warcraft and Blizzard Entertainment are trademarks or registered trademarks of Blizzard Entertainment, Inc. in the U.S. and/or other countries.

Welcome back to the second in my series of notebooks detailing the analysis of World of Warcraft population data. In our first [notebook](https://github.com/ereidelbach/wow/blob/master/warcraft1_data_collection.ipynb) we walked through the process of obtaining the data. In this notebook, we're going to dive into the data, explore it, and reshape it to fit the format we want to simply any future analysis we want to run moving forward. 

Let the fun begin!

## 1. Data Exploration

In our last notebook, we did a brief dive into the data and discovered that it is heavily nested. Let's refresh ourselves with the actual structure of the data before we go any further.

In [3]:
# Import the necessary packages
from bs4 import BeautifulSoup
import json
import os
import pandas as pd
import requests
import time

In [8]:
# create a list of all realm files in the US folder
DIR = 'data/realms-us'
list_realms_us = sorted([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
list_realms_us[-5:]

['us-ysera_08-15-2018-15:46.json',
 'us-ysondre_08-15-2018-15:46.json',
 'us-zangarmarsh_08-15-2018-15:46.json',
 'us-zuljin_08-15-2018-15:46.json',
 'us-zuluhed_08-15-2018-15:46.json']

My alliance characters just happen to be on Ysera so let's use that file for our initial analysis.

In [10]:
with open('data/realms-us/us-ysera_08-15-2018-15:46.json', 'r') as f:
    json_ysera = json.load(f)

As we said, the realm data is heavily nested so we'll save space (and memory) by avoiding outputting any of that data here.  However, the general structure of the realm data that we're interested in is as follows:
> Characters
> > Gender
> > > Class
> > > > Race
> > > > > Level
> > > > > > CharacterName